In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df = pd.read_csv("nice_zomato.csv")
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


In [3]:
df.head(10)

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,3.8/5,286,+91 9980040002\r\n+91 9980063005,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,...,0,0,0,0,0,0,0,1,1,0
6,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6/5,2556,080 48653961\r\n080 48655715,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,...,0,0,0,0,0,0,0,1,0,1
7,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0/5,324,+91 8884135549\r\n+91 9449449316,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,...,0,0,0,0,0,0,0,1,1,0
8,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2/5,504,+91 9945230807\r\n+91 9743804471,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,...,0,0,0,0,0,0,0,1,1,0
9,https://www.zomato.com/bangalore/caf%C3%A9-dow...,"12,29 Near PES University Back Gate, D'Souza N...",CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Down The A...,4.1/5,402,080 26724489\r\n+91 7406048982,Cafe,"Waffles, Pasta, Crispy Chicken, Honey Chilli C...",Cafe,500,...,0,0,0,0,0,0,0,1,1,0


In [4]:
df1 = df.dropna()
df1

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


Preprocessing😊

In [5]:
df2 = df1.drop_duplicates()
df2

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


In [6]:
print(df.drop(['url','dish_liked','phone'],axis=1))


                                                 address  \
0      942, 21st Main Road, 2nd Stage, Banashankari, ...   
1      2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...   
2      1112, Next to KIMS Medical College, 17th Cross...   
3      1st Floor, Annakuteera, 3rd Stage, Banashankar...   
4      10, 3rd Floor, Lakshmi Associates, Gandhi Baza...   
...                                                  ...   
23188  2nd Floor, Iona, Virginia Mall, Whitefield Mai...   
23189  Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...   
23190  Keys Hotel, 6, 1st Phase Industrial Area, Near...   
23191  139/C1, Next To GR Tech Park, Pattandur Agraha...   
23192  Sheraton Grand Bengaluru Whitefield Hotel & Co...   

                                                    name    rate  votes  \
0                                                  Jalsa   4.1/5    775   
1                                         Spice Elephant   4.1/5    787   
2                                        San Churro Ca

In [7]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [8]:
df.isnull().sum()
df.dropna(how='any',inplace=True)

In [9]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


In [10]:
df = df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [11]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


In [12]:
df['cost'] = df['cost'].astype(str) 
df['cost'] = df['cost'].apply(lambda x: x.replace(',','.')) 
df['cost'] = df['cost'].astype(float)

In [13]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,0,1,0,0,0,1,0,1,0


In [14]:
df = df.loc[df.rate !='NEW']
df = df.loc[df.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
df.rate = df.rate.apply(remove_slash).str.strip().astype('float')

In [15]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,0,1,0,0,0,0,1,0,1
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,0,1,0,0,0,1,0,1,0
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,0,1,0,0,0,1,0,1,0
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,0,1,0,0,0,1,0,1,0


In [16]:
df["reviews_list"] = df["reviews_list"].str.lower()



In [17]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,0,1,0,0,0,0,1,0,1
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,0,1,0,0,0,1,0,1,0
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,0,1,0,0,0,1,0,1,0
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,0,1,0,0,0,1,0,1,0


In [18]:
df.columns

Index(['url', 'address', 'name', 'rate', 'votes', 'phone', 'rest_type',
       'dish_liked', 'cuisines', 'cost',
       ...
       'location_Vijay Nagar', 'location_West Bangalore',
       'location_Whitefield', 'location_Wilson Garden', 'location_Yelahanka',
       'location_Yeshwantpur', 'online_order_No', 'online_order_Yes',
       'book_table_No', 'book_table_Yes'],
      dtype='object', length=106)

In [19]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,0,1,0,0,0,0,1,0,1
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,0,1,0,0,0,1,0,1,0
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,0,1,0,0,0,1,0,1,0
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,0,1,0,0,0,1,0,1,0


In [20]:
restaurants = list(df['name'].unique())
df['Mean Rating'] = 0


In [21]:
for i in range(len(restaurants)):
    df['Mean Rating'][df['name'] == restaurants[i]] = df['rate'][df['name'] == restaurants[i]].mean()
    

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
df[['Mean Rating']] = scaler.fit_transform(df[['Mean Rating']]).round(2)

In [23]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [24]:
import pandas as pd
mdf = pd.read_csv("nice_zomato.csv")
mdf

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Vijay Nagar,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,1,0,1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,1,1,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,1,1,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,1,0,1,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,0,0,1,0,0,0,0,1,0,1
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,0,1,0,0,0,1,0,1,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,0,1,0,0,0,1,0,1,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,0,1,0,0,0,1,0,1,0


In [25]:
mdf1 = pd.DataFrame(mdf.rest_type.str.split(',',1).tolist(), columns = ['rest_type1','rest_type2'])
mdf = mdf.join(mdf1)
#mdf.drop(['rest_type'], axis=1, inplace=True)
mdf

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,rest_type1,rest_type2
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,1,0,1,Casual Dining,None
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,1,1,0,Casual Dining,None
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,1,1,0,Cafe,Casual Dining
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,1,0,1,0,Quick Bites,None
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,1,0,1,0,Casual Dining,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,1,0,0,0,0,1,0,1,Bar,Casual Dining
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,1,0,0,0,1,0,1,0,Fine Dining,Bar
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,1,0,0,0,1,0,1,0,Casual Dining,Bar
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,1,0,0,0,1,0,1,0,Casual Dining,Bar


In [26]:
mdf3 = pd.DataFrame(mdf.dish_liked.str.split(',',6).tolist(), columns = ['liked_1','liked_2','liked_3','liked_4', 'liked_5', 'liked_6','liked_7'])
mdf = mdf.join(mdf3)
#mdf.drop(['rest_type'], axis=1, inplace=True)
mdf

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,book_table_Yes,rest_type1,rest_type2,liked_1,liked_2,liked_3,liked_4,liked_5,liked_6,liked_7
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,1,Casual Dining,None,Pasta,Lunch Buffet,Masala Papad,Paneer Lajawab,Tomato Shorba,Dum Biryani,Sweet Corn Soup
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,Casual Dining,None,Momos,Lunch Buffet,Chocolate Nirvana,Thai Green Curry,Paneer Tikka,Dum Biryani,Chicken Biryani
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,Cafe,Casual Dining,Churros,Cannelloni,Minestrone Soup,Hot Chocolate,Pink Sauce Pasta,Salsa,Veg Supreme Pizza
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,Quick Bites,None,Masala Dosa,None,None,None,None,None,None
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,Casual Dining,None,Panipuri,Gol Gappe,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200",...,1,Bar,Casual Dining,Beer,Chicken Guntur,Paneer Tikka,Fish,Noodles,Cocktails,Gulab Jamun
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,"2,000",...,0,Fine Dining,Bar,Rooftop Ambience,None,None,None,None,None,None
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200",...,0,Casual Dining,Bar,Salads,Coffee,Breakfast Buffet,Halwa,Chicken Biryani,Lunch Buffet,Noodles
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,...,0,Casual Dining,Bar,Biryani,Andhra Meal,None,None,None,None,None


In [27]:
dfregion = pd.get_dummies(mdf.rest_type1, prefix='rest_type1')
mdf = mdf.join(dfregion)
mdf.drop(['rest_type1'], axis=1, inplace=True)
mdf.head(10)

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,rest_type1_Food Court,rest_type1_Food Truck,rest_type1_Kiosk,rest_type1_Lounge,rest_type1_Mess,rest_type1_Microbrewery,rest_type1_Pub,rest_type1_Quick Bites,rest_type1_Sweet Shop,rest_type1_Takeaway
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,0,0,0
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,0,0,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,0,0,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,0,1,0,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,0,0,0,0
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,3.8/5,286,+91 9980040002\r\n+91 9980063005,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,...,0,0,0,0,0,0,0,0,0,0
6,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6/5,2556,080 48653961\r\n080 48655715,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,...,0,0,0,0,0,0,0,0,0,0
7,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0/5,324,+91 8884135549\r\n+91 9449449316,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,...,0,0,0,0,0,0,0,0,0,0
8,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2/5,504,+91 9945230807\r\n+91 9743804471,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,...,0,0,0,0,0,0,0,0,0,0
9,https://www.zomato.com/bangalore/caf%C3%A9-dow...,"12,29 Near PES University Back Gate, D'Souza N...",CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Down The A...,4.1/5,402,080 26724489\r\n+91 7406048982,Cafe,"Waffles, Pasta, Crispy Chicken, Honey Chilli C...",Cafe,500,...,0,0,0,0,0,0,0,0,0,0


In [28]:
dfregion = pd.get_dummies(mdf.rest_type2, prefix='rest_type2')
mdf = mdf.join(dfregion)
mdf.drop(['rest_type2'], axis=1, inplace=True)
mdf.head(10)

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,approx_cost(for two people),...,rest_type2_ Delivery,rest_type2_ Dessert Parlor,rest_type2_ Food Court,rest_type2_ Irani Cafee,rest_type2_ Kiosk,rest_type2_ Lounge,rest_type2_ Microbrewery,rest_type2_ Pub,rest_type2_ Quick Bites,rest_type2_ Sweet Shop
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,...,0,0,0,0,0,0,0,0,0,0
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,...,0,0,0,0,0,0,0,0,0,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,...,0,0,0,0,0,0,0,0,0,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300,...,0,0,0,0,0,0,0,0,0,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,...,0,0,0,0,0,0,0,0,0,0
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,3.8/5,286,+91 9980040002\r\n+91 9980063005,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,...,0,0,0,0,0,0,0,0,0,0
6,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6/5,2556,080 48653961\r\n080 48655715,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,...,0,0,0,0,0,0,0,0,0,0
7,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0/5,324,+91 8884135549\r\n+91 9449449316,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,...,0,0,0,0,0,0,0,0,0,0
8,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2/5,504,+91 9945230807\r\n+91 9743804471,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,...,0,0,0,0,0,0,0,0,0,0
9,https://www.zomato.com/bangalore/caf%C3%A9-dow...,"12,29 Near PES University Back Gate, D'Souza N...",CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Down The A...,4.1/5,402,080 26724489\r\n+91 7406048982,Cafe,"Waffles, Pasta, Crispy Chicken, Honey Chilli C...",Cafe,500,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#mdf.drop(['rest_type1'], axis=1, inplace=True)
#mdf.drop(['rest_type2'], axis=1, inplace=True)
mdf.drop(['rest_type'], axis=1, inplace=True)
mdf

,url,address,name,rate,votes,phone,dish_liked,cuisines,approx_cost(for two people),reviews_list,...,rest_type2_ Delivery,rest_type2_ Dessert Parlor,rest_type2_ Food Court,rest_type2_ Irani Cafee,rest_type2_ Kiosk,rest_type2_ Lounge,rest_type2_ Microbrewery,rest_type2_ Pub,rest_type2_ Quick Bites,rest_type2_ Sweet Shop
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1/5,775,080 42297555\r\n+91 9743772233,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",...,0,0,0,0,0,0,0,0,0,0
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1/5,787,080 41714161,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",...,0,0,0,0,0,0,0,0,0,0
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8/5,918,+91 9663487993,"Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",...,0,0,0,0,0,0,0,0,0,0
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7/5,88,+91 9620009302,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",...,0,0,0,0,0,0,0,0,0,0
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8/5,166,+91 8026612447\r\n+91 9901210005,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23188,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8 /5,128,+91 7625087121\n+91 8050587483,"Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean","1,200","[('Rated 3.0', ""RATED\n Nice place to hangout...",...,0,0,0,0,0,0,0,0,0,0
23189,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9 /5,77,080 49435000,Rooftop Ambience,Finger Food,"2,000","[('Rated 4.0', 'RATED\n Went there post dinne...",...,0,0,0,0,0,0,0,0,0,0
23190,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8 /5,161,080 39451000\n+91 8884038484,"Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian","1,200","[('Rated 3.0', 'RATED\n Place is good not tha...",...,0,0,0,0,0,0,0,0,0,0
23191,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5 /5,81,080 65951222,"Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800,"[('Rated 4.0', 'RATED\n A fine place to chill...",...,0,0,0,0,0,0,0,0,0,0


In [30]:
restaurants = list(df['name'].unique())
df['Mean Rating'] = 0


In [31]:

for i in range(len(restaurants)):
    df['Mean Rating'][df['name'] == restaurants[i]] = df['rate'][df['name'] == restaurants[i]].mean()
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
df[['Mean Rating']] = scaler.fit_transform(df[['Mean Rating']]).round(2)
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [32]:
df["reviews_list"] = df["reviews_list"].str.lower()

In [33]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [34]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [35]:
df["reviews_list"] = df["reviews_list"].apply(lambda text: remove_punctuation(text))

In [36]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [37]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


In [38]:
df["reviews_list"] = df["reviews_list"].apply(lambda text: remove_urls(text))


In [39]:
df[['reviews_list', 'cuisines']].sample(5)
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [40]:
restaurant_names = list(df['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

In [41]:
df

,url,address,name,rate,votes,phone,rest_type,dish_liked,cuisines,cost,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,080 42297555\r\n+91 9743772233,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,918,+91 9663487993,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,88,+91 9620009302,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,166,+91 8026612447\r\n+91 9901210005,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,"2nd Floor, Iona, Virginia Mall, Whitefield Mai...",Izakaya Gastro Pub,3.8,128,+91 7625087121\n+91 8050587483,"Bar, Casual Dining","Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,"Bengaluru Marriott Hotel, 75, 8th Road, EPIP A...",M Bar - Bengaluru Marriott Hotel Whitefield,3.9,77,080 49435000,"Fine Dining, Bar",Rooftop Ambience,Finger Food,2.0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,"Keys Hotel, 6, 1st Phase Industrial Area, Near...",Keys Cafe - Keys Hotel,2.8,161,080 39451000\n+91 8884038484,"Casual Dining, Bar","Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,2.5,81,080 65951222,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,...,0,1,0,0,0,1,0,1,0,1.61


In [42]:
df=df.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [43]:
df

,url,name,rate,phone,dish_liked,cuisines,cost,reviews_list,city,location_BTM,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,Jalsa,4.1,080 42297555\r\n+91 9743772233,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,rated 40 ratedn a beautiful place to dine int...,Banashankari,0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,Spice Elephant,4.1,080 41714161,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,rated 40 ratedn had been here for dinner with...,Banashankari,0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,San Churro Cafe,3.8,+91 9663487993,"Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience is not that good eno...,Banashankari,0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,Addhuri Udupi Bhojana,3.7,+91 9620009302,Masala Dosa,"South Indian, North Indian",300.0,rated 40 ratedn great food and proper karnata...,Banashankari,0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,Grand Village,3.8,+91 8026612447\r\n+91 9901210005,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,rated 40 ratedn very good restaurant in neigh...,Banashankari,0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,Izakaya Gastro Pub,3.8,+91 7625087121\n+91 8050587483,"Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,rated 30 ratedn nice place to hangout with ki...,Whitefield,0,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,M Bar - Bengaluru Marriott Hotel Whitefield,3.9,080 49435000,Rooftop Ambience,Finger Food,2.0,rated 40 ratedn went there post dinner just f...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,Keys Cafe - Keys Hotel,2.8,080 39451000\n+91 8884038484,"Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,rated 30 ratedn place is good not that great ...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,Bhagini,2.5,080 65951222,"Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,rated 40 ratedn a fine place to chill after o...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,1.61


Implement TF-IDF Vectorizer

In [44]:
reviews_list1="rated 40 ratedn a beautiful place to dine int"
reviews_list1="rated 30 ratedn ambience is not that good eno"
reviews_list2="rated 40 ratedn had been here for dinner with"
reviews_list2="rated 40 ratedn great food and proper karnata"

In [45]:
bagOfWordsA=reviews_list1.split(" ")
bagOfWordsB=reviews_list2.split(" ")

In [46]:
uniqueWords=set(bagOfWordsA).union(set(bagOfWordsB))

In [47]:
print(uniqueWords)

{'good', 'eno', 'rated', 'and', '30', 'that', 'karnata', 'food', 'is', 'proper', 'great', 'ambience', 'not', '40', 'ratedn'}


In [48]:
numOfWordsA = dict.fromkeys(uniqueWords,0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
numOfWordsB = dict.fromkeys(uniqueWords,0)

for word in bagOfWordsB:
    numOfWordsB[word] += 1

In [49]:
print(numOfWordsA)
print (numOfWordsB)


{'good': 1, 'eno': 1, 'rated': 1, 'and': 0, '30': 1, 'that': 1, 'karnata': 0, 'food': 0, 'is': 1, 'proper': 0, 'great': 0, 'ambience': 1, 'not': 1, '40': 0, 'ratedn': 1}
{'good': 0, 'eno': 0, 'rated': 1, 'and': 1, '30': 0, 'that': 0, 'karnata': 1, 'food': 1, 'is': 0, 'proper': 1, 'great': 1, 'ambience': 0, 'not': 0, '40': 1, 'ratedn': 1}


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [66]:
tf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tf.fit_transform(df['reviews_list'])
tfidf_matrix.shape

(23046, 154194)

In [67]:
print(tfidf_matrix)

  (0, 35947)	0.22831299334979838
  (0, 79869)	0.06807989355871753
  (0, 128019)	0.1470605562199444
  (0, 17185)	0.15577163509138406
  (0, 5724)	0.25726759714185826
  (0, 32634)	0.12021252825026373
  (0, 99722)	0.15624391799802265
  (0, 32444)	0.12713745857717906
  (0, 140236)	0.17050015362319565
  (0, 139163)	0.1804749349558954
  (0, 30483)	0.11628319608493788
  (0, 41839)	0.11857863557148168
  (0, 127333)	0.15269833894928977
  (0, 26495)	0.1589561303808515
  (0, 125411)	0.13369810628537332
  (0, 127268)	0.10067489282830648
  (0, 79698)	0.11450889058415568
  (0, 97416)	0.09553969644256363
  (0, 71132)	0.25726759714185826
  (0, 147927)	0.08193990692862951
  (0, 31803)	0.08752085257341044
  (0, 89447)	0.042291502072737776
  (0, 144326)	0.014184249402904291
  (0, 138907)	0.023038640508356238
  (0, 13866)	0.022353758968915317
  :	:
  (23045, 48218)	0.017866423936626713
  (23045, 108025)	0.022355539020834442
  (23045, 126819)	0.008978428872742706
  (23045, 149347)	0.6676235373911042
  (2304

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(['reviews_list'])

In [77]:
print(tfidf_matrix)

  (0, 0)	1.0


Cosine Similarity

In [89]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [80]:
tfidf_matrix = tfidf_matrix.astype(np.float64)
cosine_sim = cosine_similarity(tfidf_matrix)

In [82]:
print(cosine_sim )

[[1.]]


In [54]:
def recommend(name, cosine_similarities = cosine_similarities):recommend_restaurant = []
    

In [55]:
score_series = pd.Series(cosine_similarities).sort_values(ascending=False)
    

In [56]:
 top30_indexes = list(score_series.iloc[0:31].index)
    

In [57]:
print(top30_indexes)

[1, 0]


In [59]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["reviews_list"] = df["reviews_list"].apply(lambda text: remove_punctuation(text))
df

,url,name,rate,phone,dish_liked,cuisines,cost,reviews_list,city,location_BTM,...,location_West Bangalore,location_Whitefield,location_Wilson Garden,location_Yelahanka,location_Yeshwantpur,online_order_No,online_order_Yes,book_table_No,book_table_Yes,Mean Rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,Jalsa,4.1,080 42297555\r\n+91 9743772233,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,rated 40 ratedn a beautiful place to dine int...,Banashankari,0,...,0,0,0,0,0,0,1,0,1,3.99
1,https://www.zomato.com/bangalore/spice-elephan...,Spice Elephant,4.1,080 41714161,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,rated 40 ratedn had been here for dinner with...,Banashankari,0,...,0,0,0,0,0,0,1,1,0,3.97
2,https://www.zomato.com/SanchurroBangalore?cont...,San Churro Cafe,3.8,+91 9663487993,"Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience is not that good eno...,Banashankari,0,...,0,0,0,0,0,0,1,1,0,3.58
3,https://www.zomato.com/bangalore/addhuri-udupi...,Addhuri Udupi Bhojana,3.7,+91 9620009302,Masala Dosa,"South Indian, North Indian",300.0,rated 40 ratedn great food and proper karnata...,Banashankari,0,...,0,0,0,0,0,1,0,1,0,3.45
4,https://www.zomato.com/bangalore/grand-village...,Grand Village,3.8,+91 8026612447\r\n+91 9901210005,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,rated 40 ratedn very good restaurant in neigh...,Banashankari,0,...,0,0,0,0,0,1,0,1,0,3.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23041,https://www.zomato.com/bangalore/izakaya-gastr...,Izakaya Gastro Pub,3.8,+91 7625087121\n+91 8050587483,"Beer, Chicken Guntur, Paneer Tikka, Fish, Nood...","North Indian, Continental, Mediterranean",1.2,rated 30 ratedn nice place to hangout with ki...,Whitefield,0,...,0,1,0,0,0,0,1,0,1,3.58
23042,https://www.zomato.com/bangalore/m-bar-bengalu...,M Bar - Bengaluru Marriott Hotel Whitefield,3.9,080 49435000,Rooftop Ambience,Finger Food,2.0,rated 40 ratedn went there post dinner just f...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,3.71
23043,https://www.zomato.com/bangalore/keys-cafe-key...,Keys Cafe - Keys Hotel,2.8,080 39451000\n+91 8884038484,"Salads, Coffee, Breakfast Buffet, Halwa, Chick...","Chinese, Continental, North Indian",1.2,rated 30 ratedn place is good not that great ...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,2.61
23044,https://www.zomato.com/bangalore/bhagini-2-whi...,Bhagini,2.5,080 65951222,"Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,rated 40 ratedn a fine place to chill after o...,Whitefield,0,...,0,1,0,0,0,1,0,1,0,1.61


In [60]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GiNiUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

df["reviews_list"] = df["reviews_list"].apply(lambda text: remove_urls(text))

df[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
10402,rated 40 ratedn visited today for valentines ...,"Continental, North Indian, Pizza, Italian"
14648,rated 20 ratedn i was quite excited to try ka...,Kashmiri
4890,rated 10 ratedn when you try to lure customer...,"Cafe, Pizza, Fast Food, Beverages"
20719,rated 40 ratedn so another office dinner my g...,Chinese
9279,rated 40 ratedn a fan of tender coconut meets...,"Ice Cream, Desserts"


In [62]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

df["reviews_list"] = df["reviews_list"].apply(lambda text: remove_urls(text))

df[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
16182,rated 50 ratedn best place for donne biryani ...,Biryani
11142,rated 30 ratedn felt like trying chinese and ...,"Chinese, Thai, Momos, Vietnamese"
5114,rated 40 ratedn well located in the office ar...,"Burger, Turkish, Arabian, Fast Food, Salad, He..."
13431,rated 30 ratedn this outlet is close to my of...,"Beverages, Fast Food"
18796,rated 50 ratedn ive been a fan of faasos wrap...,"North Indian, Biryani, Fast Food"
